In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


Cloning into 'CD-CNN'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 82 (delta 31), reused 77 (delta 26), pack-reused 0
Unpacking objects: 100% (82/82), done.


In [2]:
from data.DataLoader import NUAADataLoader, MSUDataLoader
from training.trainModel import trainModel, benchmarkModel

In [3]:
nuaa_data_source =  "../input/nuaadataset/NUAA/"
msu_data_source =  "../input/msumsfd/MSU-MSFD/"
saved_models = "trained_models/"

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.keras.backend.clear_session()

Num GPUs Available:  1


2021-12-01 05:18:15.643105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-01 05:18:15.762561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-01 05:18:15.763462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [5]:
NUAAdtLoader = NUAADataLoader(nuaa_data_source)
NUAAdtLoader.loadData()

Found 12614 files belonging to 2 classes.
Using 10092 files for training.


2021-12-01 05:18:24.455919: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-01 05:18:24.459123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-01 05:18:24.460256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-01 05:18:24.461330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Found 12614 files belonging to 2 classes.
Using 2522 files for validation.
Found 12614 files belonging to 2 classes.
Using 10092 files for training.
Found 12614 files belonging to 2 classes.
Using 2522 files for validation.
Found 12614 files belonging to 2 classes.
Using 10092 files for training.
Found 12614 files belonging to 2 classes.
Using 2522 files for validation.


In [6]:
train_raw = NUAAdtLoader.getTrainSet('raw')
valid_raw = NUAAdtLoader.getValidSet('raw')

train_norm = NUAAdtLoader.getTrainSet('normalized')
valid_norm = NUAAdtLoader.getValidSet('normalized')

train_det = NUAAdtLoader.getTrainSet('detected')
valid_det = NUAAdtLoader.getValidSet('detected')

In [7]:
MSUdtLoader = MSUDataLoader(msu_data_source)
MSUdtLoader.preLoadData()
MSUdtLoader.loadData(train_frames=30)

In [8]:
x_train, y_train = MSUdtLoader.getTrainSet()
x_valid, y_valid = MSUdtLoader.getValidSet()

In [9]:
validation_dict = {"raw":valid_raw, "det":valid_det, "norm":valid_norm}
training_dict = {"raw":train_raw, "det":train_det, "norm":train_norm}

training_dataset = "norm" # "raw" "det" "norm"
nb_epochs = 1
Theta = 0.4

In [10]:
# Training model
if training_dataset == "msu":
    model, history = trainModel(x_train, y_train=y_train, valid=(x_valid, y_valid), ds_name='msu', epochs=nb_epochs, theta=Theta, g_dropout=False)
elif training_dataset in validation_dict.keys():
    model, history = trainModel(training_dict[training_dataset], valid=validation_dict[training_dataset], ds_name=training_dataset, epochs=nb_epochs, theta=Theta)

2021-12-01 05:19:39.950528: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-01 05:19:41.751340: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1262/1262 [==============================] - 630s 489ms/step - loss: 0.2415 - binary_accuracy: 0.8987 - precision: 0.9052 - recall: 0.8362 - val_loss: 0.1738 - val_binary_accuracy: 0.9385 - val_precision: 0.9692 - val_recall: 0.8787


2021-12-01 05:31:07.898755: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [11]:
# Or loading model
model_raw = load_model(saved_models+'model_'+training_dataset)

In [12]:
s_msu = benchmarkModel(model, x_valid, y_valid)

8/8 [==============================] - 7s 591ms/step - loss: 1.0230 - binary_accuracy: 0.8250 - precision: 0.8611 - recall: 0.7750


In [13]:
s_raw = benchmarkModel(model, valid_raw)

316/316 [==============================] - 44s 138ms/step - loss: 3.3536 - binary_accuracy: 0.6963 - precision: 0.5836 - recall: 0.9172


In [14]:
s_detected = benchmarkModel(model, valid_det)

316/316 [==============================] - 44s 137ms/step - loss: 0.4759 - binary_accuracy: 0.8914 - precision: 0.9135 - recall: 0.8133


In [15]:
s_normalized = benchmarkModel(model, valid_norm)

316/316 [==============================] - 45s 141ms/step - loss: 0.1738 - binary_accuracy: 0.9385 - precision: 0.9692 - recall: 0.8787
